In [ ]:
# import logging

# # Configure logging to see info messages
# logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')

In [ ]:
import pandas as pd
from acti_motus import Features, Activities
from pathlib import Path


files = Path('data/thigh').glob('*.parquet')

features = Features()
activities = Activities()

results = []
results_trunk = []

for thigh in files:
    gt = thigh.parent.parent / 'ground_truth' / thigh.name

    ground_truth = pd.read_parquet(gt)
    df = pd.read_parquet(thigh)

    extracted_features = features.extract(df)
    activity, references = activities.detect(extracted_features, orientation=True)

    df = ground_truth.join(activity, how='left')
    df.dropna(subset=['activity'], inplace=True)
    results.append(df)

    # Add trunk
    trunk = thigh.parent.parent / 'trunk' / thigh.name
    trunk = pd.read_parquet(trunk)

    features_trunk = features.extract(trunk)
    activity_trunk, references_trunk = activities.detect(extracted_features, trunk=features_trunk)
    trunk_df = ground_truth.join(activity_trunk, how='left')
    trunk_df.dropna(subset=['activity'], inplace=True)
    results_trunk.append(trunk_df)

results = pd.concat(results)
results.to_parquet('processed_thigh.parquet', index=True)

results_trunk = pd.concat(results_trunk)
results_trunk.to_parquet('processed_trunk.parquet', index=True)